In [1]:
import requests
import csv
from collections import deque
def get_coordinates(location, api_key):
    url = "https://geocode.search.hereapi.com/v1/geocode"
    params = {
        "q": location,
        "in": "countryCode:VNM",  # Giới hạn kết quả trong phạm vi Việt Nam (VNM là mã quốc gia ISO cho Việt Nam)
        "apiKey": api_key
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    # Kiểm tra xem có kết quả không
    if data["items"]:
        position = data["items"][0]["position"]
        return position["lat"], position["lng"]
    else:
        print("Không tìm thấy tọa độ cho địa điểm:", location)
        return None

# Các địa điểm chính với địa chỉ cụ thể hơn
api_key = "TCvst1aIZvZTk95xxKLuxb__GWCl2LTdlgTKlcs28_M"
locations = [
    "Ngã tư Hàng Xanh, Bình Thạnh, Hồ Chí Minh, Vietnam", 
    "Sân bay Tân Sơn Nhất, Tân Bình, Hồ Chí Minh, Vietnam", 
    "Ngã tư Bảy Hiền, Tân Bình, Hồ Chí Minh, Vietnam"
]

# Lấy tọa độ của các địa điểm
coordinates = [get_coordinates(location, api_key) for location in locations]
print("Tọa độ:", coordinates)
def get_route_info(origin, destination, api_key):
    url = "https://router.hereapi.com/v8/routes"
    params = {
        "transportMode": "car",
        "origin": f"{origin[0]},{origin[1]}",
        "destination": f"{destination[0]},{destination[1]}",
        "return": "summary,typicalDuration",
        "apiKey": api_key
    }
    response = requests.get(url, params=params)
    data = response.json()
    route_summary = data["routes"][0]["sections"][0]["summary"]
    return route_summary

# Tạo ma trận khoảng cách giữa các địa điểm
def build_distance_matrix(coords, api_key):
    num_points = len(coords)
    distance_matrix = [[0 for _ in range(num_points)] for _ in range(num_points)]
    time_matrix = [[0 for _ in range(num_points)] for _ in range(num_points)]

    for i in range(num_points):
        for j in range(num_points):
            if i != j:
                route_info = get_route_info(coords[i], coords[j], api_key)
                distance_matrix[i][j] = route_info["length"]  # Khoảng cách
                time_matrix[i][j] = route_info["baseDuration"]  # Thời gian di chuyển
    return distance_matrix, time_matrix

# Xây dựng ma trận
distance_matrix, time_matrix = build_distance_matrix(coordinates, api_key)
print("Ma trận khoảng cách:", distance_matrix)
print("Ma trận thời gian:", time_matrix)

def get_traffic_data_with_apikey(lat, lng, api_key, radius=500):
    url = "https://data.traffic.hereapi.com/v7/flow"
    params = {
        "in": f"circle:{lat},{lng};r={radius}",  # Khu vực bán kính xung quanh tọa độ
        "apiKey": api_key,
        "locationReferencing": "shape"  # Yêu cầu trả về dữ liệu shape
    }
    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        
        jam_factors = []
        for rws in data.get("RWS", []):
            for rw in rws.get("RW", []):
                for fis in rw.get("FIS", []):
                    for fi in fis.get("FI", []):
                        shape = fi.get("SHP", [])  # Hình dạng đoạn đường
                        for cf in fi.get("CF", []):
                            jam_factor = cf.get("JF", None)
                            if jam_factor is not None:
                                jam_factors.append(jam_factor)
        
        # Trả về JF trung bình hoặc giá trị nào đó đại diện cho toàn bộ khu vực
        return sum(jam_factors) / len(jam_factors) if jam_factors else None
    else:
        print("Error:", response.status_code, response.json())
        return None

# Xây dựng ma trận lưu lượng giao thông
def build_traffic_matrix(coords, api_key):
    num_points = len(coords)
    traffic_matrix = [[0 for _ in range(num_points)] for _ in range(num_points)]

    for i in range(num_points):
        lat, lng = coords[i]
        jam_factor = get_traffic_data_with_apikey(lat, lng, api_key)
        
        for j in range(num_points):
            if i != j:
                traffic_matrix[i][j] = max(1, int(10 - (jam_factor or 5)) * 100)  # Giới hạn lưu lượng dựa trên JF
    
    return traffic_matrix

# Tạo ma trận lưu lượng
traffic_matrix = build_traffic_matrix(coordinates, api_key)
print("Ma trận lưu lượng giao thông:", traffic_matrix)





def save_matrix_to_csv(matrix, filename):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(matrix)

# Lưu các ma trận vào file CSV
save_matrix_to_csv(distance_matrix, "distance_matrix.csv")
save_matrix_to_csv(time_matrix, "time_matrix.csv")
save_matrix_to_csv(traffic_matrix, "traffic_matrix.csv")



def bfs(graph, source, sink, parent):
    visited = [False] * len(graph)
    queue = deque([source])
    visited[source] = True
    
    while queue:
        u = queue.popleft()
        
        for v, capacity in enumerate(graph[u]):
            if not visited[v] and capacity > 0:  # Nếu đỉnh chưa được thăm và còn lưu lượng
                queue.append(v)
                visited[v] = True
                parent[v] = u
                if v == sink:
                    return True
    return False

def edmonds_karp(graph, source, sink):
    parent = [-1] * len(graph)  # Danh sách lưu đường đi từ nguồn đến đích
    max_flow = 0
    
    while bfs(graph, source, sink, parent):
        path_flow = float('Inf')
        s = sink
        
        # Tìm lưu lượng nhỏ nhất trên đường đi tìm được
        while s != source:
            path_flow = min(path_flow, graph[parent[s]][s])
            s = parent[s]
        
        # Cập nhật lưu lượng của các cạnh và các cạnh ngược lại
        v = sink
        while v != source:
            u = parent[v]
            graph[u][v] -= path_flow
            graph[v][u] += path_flow
            v = parent[v]
        
        # Thêm lưu lượng của đường đi vào tổng lưu lượng
        max_flow += path_flow
    
    return max_flow

# Chọn nguồn và đích (ví dụ: đỉnh đầu tiên là nguồn, đỉnh cuối cùng là đích)
source = 0
sink = len(traffic_matrix) - 1

# Sao chép traffic_matrix để giữ nguyên dữ liệu gốc
import copy
graph = copy.deepcopy(traffic_matrix)

# Tính dòng cực đại giữa nguồn và đích
max_flow = edmonds_karp(graph, source, sink)
print("Dòng cực đại giữa nguồn và đích:", max_flow)


Tọa độ: [(10.80131, 106.71131), (10.81447, 106.6651), (10.79283, 106.65348)]
Ma trận khoảng cách: [[0, 8651, 8226], [8150, 0, 4068], [7148, 3691, 0]]
Ma trận thời gian: [[0, 911, 1079], [1117, 0, 607], [976, 485, 0]]
Ma trận lưu lượng giao thông: [[0, 500, 500], [500, 0, 500], [500, 500, 0]]
Dòng cực đại giữa nguồn và đích: 1000
